In [3]:
import cv2
import base64
import requests
import numpy as np
import time

# --- Settings ---
VIDEO_PATH = "Test1.mp4"
OUTPUT_PATH = "tracked_with_optical_flow_and_retry.mp4"
API_URL = "https://detect.roboflow.com/nfl-object-tracking/4?api_key=RsIHVYfJftsspBQBQPkK&confidence=0.2"
RETRY_DETECTION = True

# Open video
cap = cv2.VideoCapture(VIDEO_PATH)
width = int(cap.get(3))
height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, 20.0, (width, height))

# Optical flow parameters
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Read first frame
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Step 1: Try to detect the ball in the first few frames
p0 = None
for _ in range(5):
    _, buffer = cv2.imencode('.jpg', old_frame)
    b64 = base64.b64encode(buffer).decode('utf-8')
    response = requests.post(API_URL, data=b64, headers={"Content-Type": "application/x-www-form-urlencoded"})

    try:
        preds = response.json().get("predictions", [])
        for pred in preds:
            if pred["class"] == "Ball":
                x, y = int(pred["x"]), int(pred["y"])
                p0 = np.array([[[x, y]]], dtype=np.float32)
                print(f"✅ Initial detection: ({x}, {y})")
                break
    except:
        pass

    if p0 is not None:
        break
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

if p0 is None:
    print("❌ Failed to detect the ball in initial frames.")
    cap.release()
    out.release()
    exit()

# Step 2: Start tracking with fallback
frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Track motion
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    if st[0][0] == 1:
        x, y = int(p1[0][0]), int(p1[0][1])
        cv2.circle(frame, (x, y), 10, (0, 255, 255), -1)
        cv2.putText(frame, "Tracked Ball", (x + 10, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        p0 = p1  # Update point
    else:
        cv2.putText(frame, "Tracking Lost", (30, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Step 3: Retry detection if tracking is lost
        if RETRY_DETECTION and frame_count % 5 == 0:  # Try every 5 frames
            print("🔁 Re-attempting detection...")
            _, buffer = cv2.imencode('.jpg', frame)
            b64 = base64.b64encode(buffer).decode('utf-8')
            response = requests.post(API_URL, data=b64, headers={"Content-Type": "application/x-www-form-urlencoded"})
            try:
                preds = response.json().get("predictions", [])
                for pred in preds:
                    if pred["class"] == "Ball":
                        x, y = int(pred["x"]), int(pred["y"])
                        p0 = np.array([[[x, y]]], dtype=np.float32)


SyntaxError: incomplete input (2372907260.py, line 91)